In [ ]:
import boto3
import pandas as pd
from itertools import batched

In [5]:
df = pd.read_csv('amazon_kitchenware.csv')

In [14]:
# convert column names to snake_case
df.columns = (df.columns
                .str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True)
                .str.lower()
             )

df = df.rename(columns={'asin': 'product_id'})

info_df = df[['product_id', 'title', 'brand', 'description', 'stars', 'reviews_count', 'bread_crumbs', 'url']]
price_df = df[['product_id', 'price/currency', 'price/value']]

### Writing product info into AWS Dynamodb

In [18]:
aws_admin_session = boto3.Session(profile_name='admin')

dynamodb = aws_admin_session.client('dynamodb', region_name = 'ap-southeast-1')

for batched_rows in batched(info_df.to_dict('records'), 10):
    response = dynamodb.batch_write_item(
        RequestItems={
            'product_info': [
                {
                    'PutRequest': {
                        'Item': {
                            k: {'S': str(v)} for k, v in row.items()
                        }
                    }
                } for row in batched_rows
            ]
        }
    )
    